<a href="https://colab.research.google.com/github/eclipseeyo/practiceML/blob/main/LogReg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Логистическая регрессия

Обучим логистическую регрессию для предсказания того, откликнется клиент  
на рекламное предложение (target = 1) или нет (target = 0).

## Подлючение библиотек, загрузка и обзор данных

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

In [2]:
data = pd.read_csv('https://raw.githubusercontent.com/evgpat/edu_stepik_practical_ml/main/datasets/clients_data.csv')

In [3]:
data.head()

,AGE,SOCSTATUS_WORK_FL,SOCSTATUS_PENS_FL,GENDER,CHILD_TOTAL,DEPENDANTS,PERSONAL_INCOME,LOAN_NUM_TOTAL,LOAN_NUM_CLOSED,LOAN_DLQ_NUM,TARGET
0,49,1,0,1,2,1,5000.0,1,1,2,0
1,32,1,0,1,3,3,12000.0,1,1,1,0
2,52,1,0,1,4,0,9000.0,2,1,0,0
3,39,1,0,1,1,1,25000.0,1,1,3,0
4,30,1,0,0,0,0,12000.0,2,1,2,0


In [4]:
from sklearn.model_selection import train_test_split

X = data.drop('TARGET', axis=1)
y = data['TARGET']

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, train_size=0.7, random_state=123)

## Практика

**Задание**  
Выведите на экран количество объектов каждого класса. Сколько процентов объектов относятся к положительному классу?  
Ответ округлите до целого числа (например, если доля объектов положительного класса равна 0.412, в ответ запишите 41,  
имея в виду 41 процент).

In [18]:
y.value_counts(normalize=True) * 100

,proportion
TARGET,
0,88.096959
1,11.903041


Обучим логистическую регрессию с параметрами по умолчанию.

In [19]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()

In [22]:
lr.fit(Xtrain, ytrain)

prediction = lr.predict(Xtest)

Метрику accuracy не стоит использовать при сильном дисбалансе классов. Поэтому посчитайте f1_score для оценки качества модели на тестовых данных.

f1_score принимает значения от 0 до 1. Чем ближе к 1, тем лучше модель.

In [36]:
from sklearn.metrics import f1_score

# ваш код здесь
f1_score(ytest, prediction)

0.0

**Вопрос**  
Чему равен `f1_score`?

Удивительно, да?

Давайте разберемся, почему качество такое низкое.

Предскажем вероятности классов с помощью обученной логистической регрессии на тестовых данных.

In [37]:
probs_test = lr.predict_proba(Xtest)

probs_test[:10]

array([[0.93860052, 0.06139948],
       [0.90059186, 0.09940814],
       [0.71082803, 0.28917197],
       [0.78458585, 0.21541415],
       [0.87089445, 0.12910555],
       [0.9585094 , 0.0414906 ],
       [0.93632178, 0.06367822],
       [0.96561689, 0.03438311],
       [0.94889185, 0.05110815],
       [0.93091044, 0.06908956]])

По вероятностям видно, что вероятности отнесения к положительному классу очень низкие.

Попробуем изменить порог для перевода вероятности в классы.

**Вопрос**  
Чему равен `f1_score`, если все объекты с вероятностью не меньшей 0.1, относить к положительному классу?

Ответ округлите до сотых.

In [38]:
f1_score(ytest, probs_test[:,1] >= 0.1)

0.2549336784212229

У обученной модели можно посмотреть веса (как и в линейной регрессии).  
Выведем на экран веса модели (`model.coef_`, `model.intercept_`).

In [39]:
lr.coef_, lr.intercept_

(array([[-4.61786358e-02, -1.47051283e-01, -8.61126244e-02,
         -1.54075703e-01,  1.91465958e-01, -1.41678590e-01,
          2.01720015e-05, -6.01039642e-02, -2.31268961e-01,
          2.92922844e-01]]),
 array([-0.24167938]))

Создайте `pd.DataFrame`, где в первом столбце стоят названия признаков, а во втором - их веса (так удобнее анализировать результат).  
Отсортируйте таблицу по убыванию весов.

**Вопрос**  
Какой признак имеет наибольший положительный вес?

In [61]:
#weights_df = pd.DataFrame({'feature' : Xtrain.columns, 'weight' : lr.coef_[0]})
#weights_df.reindex(weights_df['weight'].sort_values(ascending=False).index)

coefdf = pd.DataFrame(index = X.columns, data = lr.coef_[0], columns = ['coef'])

coefdf.sort_values('coef', ascending = False)

,coef
LOAN_DLQ_NUM,0.292923
CHILD_TOTAL,0.191466
PERSONAL_INCOME,0.000020
AGE,-0.046179
LOAN_NUM_TOTAL,-0.060104
SOCSTATUS_PENS_FL,-0.086113
DEPENDANTS,-0.141679
SOCSTATUS_WORK_FL,-0.147051
GENDER,-0.154076
LOAN_NUM_CLOSED,-0.231269


## Бонус

**Задание 1**  
Подберите порог для перевода вероятностей в классы, дающий максимальное значение `f1_score`.

In [ ]:
# ваш код здесь

**Задание 2**  
Во вложенном цикле подберите одновременно коэффициент регуляризации `C` у логистической регрессии и порог для перевода вероятностей в классы, дающие максимальное значение `f1_score`.

По-хорошему, чтобы не переобучиться, эти величины надо подбирать не по тесту, а по отдельной выборке.

Поэтому разобъем данные изначально на три части: `Xtrain`, `Xval`, `Xtest`.

*   В цикле при подборе `С` и порога будем обучаться по `Xtrain`, а предсказывать и измерять качество по `Xval`.

*   Качество итоговой модели с найденными `C` и порогом измерьте по `Xtest`.

Так не переобучимся!


In [ ]:
Xtrain_new, Xval, ytrain_new, yval = train_test_split(Xtrain, ytrain, train_size=0.7, random_state=123)

# ваш код для подбора C и порога здесь

In [ ]:
model = LogisticRegression(C = ...)

model.fit(Xtrain, ytrain) # обучаемся на всех тренировочных данных

prediction = model.predict_proba(Xtest)

classes = ... # переведите полученные вероятности в классы по найденному порогу

In [ ]:
# вычислите значение f1_score на тестовых данных